#### Importando as bibliotecas necessárias.

In [ ]:
import pandas as pd
import glob

#Gráficos
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go

### Agrupando as tabelas

##### Criando uma estrutura de DataFrame vazia.

In [ ]:
df_ceaps = pd.DataFrame()

##### Utilizando o módulo glob para agrupar todas as tabelas em um único DataFrame.

In [ ]:
for sheet in glob.glob("C:\\Users\\Andre\\Documents\\Estudos - TI\\7DaysOfCode\\base\\*.csv"):
    df_ = pd.read_csv(sheet,delimiter = ';',encoding='latin-1',header=1)
    df_ceaps = pd.concat([df_ceaps,df_])

In [ ]:
df_ceaps.head()

### Data Cleaning

#### Verificando o formato dos campos se estão corretos

In [ ]:
print(df_ceaps.info())

O campo de valor e data não estão no formato correto

#### Convertendo para float o campo valor

In [ ]:
df_ceaps['VALOR_REEMBOLSADO'] = df_ceaps['VALOR_REEMBOLSADO'].apply(lambda x: str(x).replace(",","."))

In [ ]:
df_ceaps['VALOR_REEMBOLSADO'] = df_ceaps['VALOR_REEMBOLSADO'].astype('float64')

#### Convertendo para DateTime o campo Data

Utilizando uma coluna auxiliar para validar os dados com formatos errados

In [ ]:
df_ceaps['DATA_NEW'] = pd.to_datetime(df_ceaps['DATA'],format='%d/%m/%Y', errors='coerce')
# O parâmetro “errors=coerce” inclui o valor NaT nas datas com o valor incorreto

In [ ]:
df_ceaps[df_ceaps['DATA_NEW'].isnull()] #Exibe os dados vazios, ou seja, com valores incorretos

In [ ]:
df_ceaps['DATA'] = df_ceaps['DATA'].replace('31/01/0219','31/01/2019')
df_ceaps['DATA'] = df_ceaps['DATA'].replace('05/05/0202','05/05/2020')
df_ceaps['DATA'] = df_ceaps['DATA'].replace('29/10/0202','29/10/2020')

In [ ]:
df_ceaps['DATA'] = pd.to_datetime(df_ceaps['DATA'], format="%d/%m/%Y")

#### Verificando se os dados foram corrigidos

In [ ]:
df_ceaps.info()

In [ ]:
df_ceaps = df_ceaps.drop(['DATA_NEW'],axis=1) #Excluindo a coluna auxiliar

In [ ]:
df_ceaps['DATA_MES'] = df_ceaps['DATA'].dt.to_period('M').astype(str)

In [ ]:
df_ceaps[df_ceaps['DATA_MES'] < '2019-01']

#### Verificando campos com valores nulos

In [ ]:
df_ceaps.isnull().sum()

Analisando o campo Documento, não me diz muita coisa e optei por excluir

In [ ]:
df_ceaps = df_ceaps.drop(['DOCUMENTO'],axis=1)

In [ ]:
df_ceaps['DETALHAMENTO'] = df_ceaps['DETALHAMENTO'].fillna('Nao Informado')

In [ ]:
df_ceaps.isnull().sum()

#### Verificando linhas duplicadas

In [ ]:
df_ceaps.duplicated().any()

#### Verificando erros de formatação no CNPJ e CPF

In [ ]:
validacao_CNPJ_CPF = df_ceaps['CNPJ_CPF'].str.len()

In [ ]:
validacao_CNPJ_CPF.value_counts()

Verificando se existe letras no campo

In [ ]:
validacao_CNPJ_CPF = df_ceaps[df_ceaps['CNPJ_CPF'].str.contains('[a-zA-Z]')]
#'str.contains('[a-zA-Z]')' Retorna uma correspondência para qualquer caractere em ordem alfabética entre a e z, minúsculas ou maiúsculas

In [ ]:
validacao_CNPJ_CPF[validacao_CNPJ_CPF == True]

#### Tratamento do campo TIPO_DESPESA

In [ ]:
df_ceaps.TIPO_DESPESA.unique()

Visando facilitar a visualização dos gráficos, substitui por nomes reduzidos

In [ ]:
tipo_despesa_tratado ={'Aluguel de imóveis para escritório político, compreendendo despesas concernentes a eles.':'Aluguel de Imoveis',
       'Divulgação da atividade parlamentar':'Divulgacao Atividade',
       'Aquisição de material de consumo para uso no escritório político, inclusive aquisição ou locação de software, despesas postais, aquisição de publicações, locação de móveis e de equipamentos. ':'Aquisicao Material',
       'Passagens aéreas, aquáticas e terrestres nacionais':'Passagens',
       'Contratação de consultorias, assessorias, pesquisas, trabalhos técnicos e outros serviços de apoio ao exercício do mandato parlamentar':'Contratacao Servicos',
       'Locomoção, hospedagem, alimentação, combustíveis e lubrificantes':'Locomocao',
       'Serviços de Segurança Privada':'Seguranca'}

In [ ]:
df_ceaps['TIPO_DESPESA_TRATADO'] = df_ceaps['TIPO_DESPESA'].map(tipo_despesa_tratado)

In [ ]:
df_ceaps['TIPO_DESPESA_TRATADO'].unique()

In [ ]:
#df_ceaps.to_excel("C:\\Users\\Andre\\Documents\\Data Studies\\7DaysOfCode\\dataset.ceaps.xlsx", index=False, header=True)

## Data Visualization

#### Verificando outiliers

Uma das melhores formas de identificar dados outliers é utilizando gráficos.

In [ ]:
px.box(df_ceaps, y='VALOR_REEMBOLSADO')

In [ ]:
df_ceaps = df_ceaps[df_ceaps['VALOR_REEMBOLSADO']<=2500]
px.box(df_ceaps, y='VALOR_REEMBOLSADO')

In [ ]:
df_senador_top10 = df_ceaps.groupby('SENADOR')['VALOR_REEMBOLSADO'].sum().sort_values(ascending=[False])[:10].reset_index()

In [ ]:
fig = px.bar(df_senador_top10, x='SENADOR',y='VALOR_REEMBOLSADO', 
             title="Top 10 senadores com mais despesas entre 2019-2022",
            width=600, height=400)
fig.update_xaxes(tickangle=45)
fig.show()